<a href="https://colab.research.google.com/github/suryansh29/DL-project/blob/main/llm-fintuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets
!pip install trl
!pip install evaluate



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 11.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-man

In [ ]:
from datasets import load_dataset, ClassLabel

mnli = load_dataset("nyu-mll/multi_nli")

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")
model = AutoModelForSequenceClassification.from_pretrained("facebook/opt-350m", num_labels=len(tokenizer))

Some weights of OPTForSequenceClassification were not initialized from the model checkpoint at facebook/opt-350m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
PATTERN="{text1} {text2} ?"
TARGET_PREFIX=" "

In [ ]:
target_tokens_ids = tokenizer.convert_tokens_to_ids(['Yes', 'No'])


In [ ]:
from datasets import ClassLabel

mnli = mnli.filter(lambda example: example["label"] != 1 and example["label"] != -1)

# change labels of contradiction examples from 2 to 1
def change_label(example):
    example["label"] = 1 if example["label"] == 2 else example["label"]
    return example
mnli = mnli.map(change_label)





{0, 1}

In [ ]:
mnli
 # change features to reflect the new labels
features = mnli["train"].features.copy()
features["label"] = ClassLabel(num_classes=len(tokenizer))
mnli = mnli.cast(features)  # overwrite old features

Casting the dataset:   0%|          | 0/261802 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/6692 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/6703 [00:00<?, ? examples/s]

In [ ]:
def preprocess_function_without_context(examples):
  pattern_examples = [
          PATTERN.format(
              text1=examples['premise'][idx],
              text2=examples['hypothesis'][idx]) + TARGET_PREFIX
          for idx in range(len(examples['premise']))
      ]

  args = (pattern_examples,)
  result = tokenizer(*args, padding="max_length", truncation=True)

# Get tokens
  result["input_tokens"] = [tokenizer.convert_ids_to_tokens(
      ids) for ids in result["input_ids"]]

  # Decode input
  result["input_text"] = [tokenizer.decode(
      ids) for ids in result["input_ids"]]

  result["label"] = [target_tokens_ids[l] for l in examples["label"]]

  return result


In [ ]:
tokenized_mnli_without_context = mnli.map(preprocess_function_without_context, batched=True)

In [ ]:
import numpy as np

def create_few_shot_examples(examples, num_shots=3):
  indices = np.random.choice(range(len(examples['premise'])), size=num_shots, replace=False)
  separate_shots_by = "\n\n"
  context=""
  for idx in indices:
    formated_sample = PATTERN.format(
        text1=examples['premise'][idx],
        text2=examples['hypothesis'][idx]
    )
    verbalized_label = tokenizer.convert_ids_to_tokens([(examples['label'][idx])])[0]
    context += f"{formated_sample}{TARGET_PREFIX}{verbalized_label}{separate_shots_by}"
  return context



def preprocess_function_with_few_shot_context(examples):
  examples["label"] = [target_tokens_ids[l] for l in examples["label"]]
  pattern_examples = [
          create_few_shot_examples(examples, 3) +
          PATTERN.format(
              text1=examples['premise'][idx],
              text2=examples['hypothesis'][idx]) + TARGET_PREFIX
          for idx in range(len(examples['premise']))
      ]
  args = (pattern_examples,)
  result = tokenizer(*args, padding="max_length", truncation=True)

# Get tokens
  result["input_tokens"] = [tokenizer.convert_ids_to_tokens(
      ids) for ids in result["input_ids"]]

  # Decode input
  result["input_text"] = [tokenizer.decode(
      ids) for ids in result["input_ids"]]

  result["label"] = examples["label"]

  return result


In [ ]:
tokenized_mnli_validation_matched_with_context = mnli['validation_matched'].map(preprocess_function_with_few_shot_context, batched=True)
tokenized_mnli_validation_mismatched_with_context = mnli['validation_mismatched'].map(preprocess_function_with_few_shot_context, batched=True)
tokenized_mnli_training_with_context = mnli['training'].map(preprocess_function_with_few_shot_context, batched=True)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

pattern_examples:  ["i don't know um do you do a lot of camping I know exactly. ? No\n\nThe bhakti movement of the Tamils brought a new warmth to the hitherto rigid Brahmanic ritual of Hinduism. The Tamils' bhakti movement froze the previously warm ritual of Hinduism. ? No\n\nyeah i i think my favorite restaurant is always been the one closest  you know the closest as long as it's it meets the minimum criteria you know of good food My favorite restaurants are always at least a hundred miles away from my house.  ? No\n\nThis site includes a list of all award winners and a searchable database of Government Executive articles. The Government Executive articles housed on the website are not able to be searched. ? ", "The bhakti movement of the Tamils brought a new warmth to the hitherto rigid Brahmanic ritual of Hinduism. The Tamils' bhakti movement froze the previously warm ritual of Hinduism. ? No\n\nuh i don't know i i have mixed emotions about him uh sometimes i like him but at the sam

In [ ]:
 mnli['train'].map(preprocess_function_without_context, batched=True)

Map:   0%|          | 0/261802 [00:00<?, ? examples/s]

Dataset({
    features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label', 'input_ids', 'attention_mask', 'input_tokens', 'input_text'],
    num_rows: 261802
})

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
hans = load_dataset("hans")


Generating train split:   0%|          | 0/30000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/30000 [00:00<?, ? examples/s]

In [ ]:
 # change features to reflect the new labels
features = hans["train"].features.copy()
features["label"] = ClassLabel(num_classes=len(tokenizer))
hans = hans.cast(features)  # overwrite old features

Casting the dataset:   0%|          | 0/30000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/30000 [00:00<?, ? examples/s]

In [ ]:
tokenized_hans_without_context = hans.map(preprocess_function_without_context, batched=True)

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=None,
    eval_dataset=tokenized_mnli_without_context['validation_matched'],
    tokenizer=tokenizer,
    # data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

HfHubHTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/repos/create (Request ID: Root=1-662d020e-1cea8134288fae8524c025ae;aad05483-a625-4353-83b7-bb3066bf6213)

Invalid username or password.


For each, calculate in-domain accuracy (on mnli) and out-domain accuracy (on hans)

1) Try out acuracy for zero shot inference without fine-tuning
2) Try out accuracy for few shot inference (in-context learning) without fine-tuning
3) Try out LoRA, QLoRA and PEFT based fine-tuning
4) KL divergence loss based fine-tuning using LoRA/QLoRa, PEFT